##### Exploratory Data Analysis of:
# Top Tech Employees' Workplace Experiance
___

**Context:** Over 67k employee reviews for Google, Amazon, Facebook, Apple, and Microsoft.

**Acknowledgements:** This data was scraped from Glassdoor.

**Questions**
- What company has been improving best?
- based on this data what company seems the best choice for a specific postion?
- is there a relationship base on stock market mometoum and volatility and employee's workplace rating?
- What pros each company is best at?
- what positions seem to be happiest in each company? (and overall)
___

Scratch Notes

- Exploratory Data Analysis and Visualization for Text Data
- Create a stunning visualization
- focus of the project is on exploratory data analysis
- worthy of posting it to Medium, Kaggle
- Emplopyers and My Tech Friends value it

https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a

https://www.kaggle.com/aashita/exploratory-data-analysis-of-comments-on-nyt

https://www.kaggle.com/petersunga/google-amazon-facebook-employee-reviews

Table of contents:
1. [Importing modules and loading data ](#import)

## Importing modules and loading data <a name="import"></a>

1. First we import python modules and setting notebook environment

In [1]:
# Essential Data Analysis Ecosystem Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# From Python Standard Library
import os  # os file operations
import re  # regular experesion  for data cleaning
# import json  # encoding and decoding json data

# import ast # Safely evaluate an expression node or a string containing a Python literal or container display.
# from pandas.io.json import json_normalize

# # Wrangling Tools
# import requests  # Python HTTP for Humans! - Version: 2.21.0
# import tweepy  # Popular Twitter library for python. -  Version: 3.5.0

In [2]:
# set plots to be embedded inline
%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

2. import the dataset

In [3]:
# dataset starting file(s)
dataset_path = 'input/'
print ('Dataset file(s):', os.listdir(dataset_path))

# TODO: Programmatically download the dataset zip file from Kaggle.

df = pd.read_csv(os.path.join(dataset_path, 'employee_reviews.csv'))

Dataset file(s): ['employee_reviews.csv']


## First look at the data:<a name="import"></a>

#### Let's look at shape of our data and 3 random observations (reviews):

In [4]:
print('Number of rows (reviews) and columns:', df.shape)
df.sample(3)

Number of rows (reviews) and columns: (67529, 17)


,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
8058,8059,amazon,"Seattle, WA","Nov 25, 2018",Current Employee - PM,Toxic is an understatement.,-Ability to work on projects that impact a lot...,-Extremely toxic work environment where manage...,-Show that you actually care about employees -...,2.0,4.0,1.0,1.0,2.0,1.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
10063,10064,amazon,"Chester, VA","Jul 18, 2018",Former Employee - ICQA,They will cut corners until theres none left t...,Its typical warehouse work (dont expect typica...,I was diagnosed with Generalized Anxiety Disor...,You guys dont understand what the workers issu...,1.0,1.0,1.0,1.0,1.0,1.0,2,https://www.glassdoor.com/Reviews/Amazon-Revie...
21635,21636,amazon,"Philadelphia, PA","Jul 28, 2015",Current Employee - Human Resources Business Pa...,"Good Company, still finding its feet",+Awesome opportunities due to growth +Your voi...,-Most of the time you are drinking from a fire...,Find your balance. Invest in the FC network wi...,3.0,1.0,3.0,5.0,4.0,3.0,2,https://www.glassdoor.com/Reviews/Amazon-Revie...


#### Information about columns by data provider:

This dataset contains employee reviews separated into the following categories. 
> Comments in blue are data tidiness and quality issues that we will fix before start exploring the data.

##### Columns
- Index: index
- - <font color='blue'> Unnamed column is redundant. Same as index. To be dropped</font>
- Company: Company name
- Location : This dataset is global, as such it may include the country's name in parenthesis i.e "Toronto, ON(Canada)". However, if the location is in the USA then it will only include the city and state i.e "Los Angeles, CA".
- - <font color='blue'> Tidiness issue: location column contains 3 different 'city, state, and country' values. This column is to be separated in 3 columns.</font>
- Date Posted: in the following format MM DD, YYYY
- Job-Title: This string will also include whether the reviewer is a 'Current' or 'Former' Employee at the time of the review
- - <font color='blue'> Tidiness issue: employee_status (current or former) and job_title are to be seperated in two columns.</font>
- Summary: Short summary of employee review
- Pros: Pros
- Cons: Cons
- Overall Rating: 1-5
- Work/Life Balance Rating: 1-5
- Culture and Values Rating: 1-5
- Career Opportunities Rating: 1-5
- Comp & Benefits Rating: 1-5
- Senior Management Rating: 1-5
- Helpful Review Count: A count of how many people found the review to be helpful
- Link to Review : This will provide you with a direct link to the page that contains the review. However it is likely that this link will be outdated

NOTE: 'none' is placed in all cells where no data value was found.



#### Information about the columns' names, datatypes, unique values, and null values:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67529 entries, 0 to 67528
Data columns (total 17 columns):
Unnamed: 0                    67529 non-null int64
company                       67529 non-null object
location                      67529 non-null object
dates                         67529 non-null object
job-title                     67529 non-null object
summary                       67409 non-null object
pros                          67529 non-null object
cons                          67529 non-null object
advice-to-mgmt                67232 non-null object
overall-ratings               67529 non-null float64
work-balance-stars            67529 non-null object
culture-values-stars          67529 non-null object
carrer-opportunities-stars    67529 non-null object
comp-benefit-stars            67529 non-null object
senior-mangemnet-stars        67529 non-null object
helpful-count                 67529 non-null int64
link                          67529 non-null object
dtypes: 

In [6]:
df.nunique()

Unnamed: 0                    67529
company                           6
location                       2044
dates                          3824
job-title                      8308
summary                       42649
pros                          66085
cons                          66049
advice-to-mgmt                35190
overall-ratings                   5
work-balance-stars               10
culture-values-stars              6
carrer-opportunities-stars       10
comp-benefit-stars               10
senior-mangemnet-stars           10
helpful-count                    99
link                           6754
dtype: int64

#### Data Preprocessing
- Unnamed column is index same as DataFrame index; To be dropped

In [7]:
df.location.value_counts()[:10]

none                    25085
Redmond, WA              5613
Seattle, WA              4963
Hyderabad (India)        2150
Mountain View, CA        1706
Bengaluru (India)        1607
Cupertino, CA            1291
New York, NY             1152
London, England (UK)      930
Menlo Park, CA            805
Name: location, dtype: int64

In [8]:
df.location.sample(30)

40859                            none
18684                       Tampa, FL
53675                     Redmond, WA
64973                 Moscow (Russia)
43233                            none
8910                             none
52773                      Frisco, TX
64957    Dublin, Co. Dublin (Ireland)
18456                     Lebanon, TN
31648            Toronto, ON (Canada)
40456                        Novi, MI
37937                 Kansas City, MO
47223                            none
3673                Mountain View, CA
64615           Singapore (Singapore)
50169                     Redmond, WA
11183                 Saint Cloud, MN
57373                     Redmond, WA
5122                             none
60839                     Redmond, WA
26673                            none
9768                    Baltimore, MD
3375                     New York, NY
63520                            none
12520                Murfreesboro, TN
32229               Bengaluru (India)
53058       

In [9]:
import re


In [10]:
# str.contains(pat, case=True, flags=0, na=nan, regex=True)[source]¶
df.location.sample(20)

52305                     Redmond, WA
25879                 Chennai (India)
35828    Dublin, Co. Dublin (Ireland)
38451                            none
49989                     Redmond, WA
49857                            none
63351                            none
23900              Campbellsville, KY
48233                            none
39717                            none
57196                     Seattle, WA
54156                            none
56538                            none
64638                            none
19466                            none
12311                     Seattle, WA
30105               Hyderabad (India)
6834     Dublin, Co. Dublin (Ireland)
53437                            none
12880                     Seattle, WA
Name: location, dtype: object

In [11]:
btween_parentheses = r'\(([^)]+)\)'  # Regular expression to get a string between parentheses


def get_country(location):
    """Extracts and returns country name from location string"""
    if location == 'none':
        return np.nan
    not_usa = re.findall(btween_parentheses, location)
    if not_usa:
        return not_usa[0]
    else:
        return 'USA'
    

def get_state(location):
    """Extracts and returns state name (if aby) from location string"""
    if location == 'none':
        return np.nan
    not_usa = re.findall(btween_parentheses, location)
    if not_usa:
        if ',' in location:
            return location.split(',')[1].split()[0]
        else:
            return np.nan
    else:
        return location.strip()[-2:]

    
def get_city(location):
    """Extracts and returns city name from location string"""
    if location == 'none':
        return np.nan
    not_usa = re.findall(btween_parentheses, location)
    if not_usa:
        if ',' in location:
            return location.split(',')[0]
        else:
            return location.split()[0]
    else:
        return location.split(',')[0]    

In [12]:
# Tidy location column to 3 columns, City, State (only for USA and some countries), and Country.
df['country'] = df.location.apply(get_country)
df['state'] = df.location.apply(get_state)
df['city'] = df.location.apply(get_city)
# Frop no longer needed location column
df.drop(columns='location', axis=1, inplace=True)

In [13]:
df.nunique()

Unnamed: 0                    67529
company                           6
dates                          3824
job-title                      8308
summary                       42649
pros                          66085
cons                          66049
advice-to-mgmt                35190
overall-ratings                   5
work-balance-stars               10
culture-values-stars              6
carrer-opportunities-stars       10
comp-benefit-stars               10
senior-mangemnet-stars           10
helpful-count                    99
link                           6754
country                         106
state                           154
city                           1842
dtype: int64

In [14]:
df[df.country == 'Iran']

,Unnamed: 0,company,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link,country,state,city
6466,6467,google,"Feb 26, 2016",Former Employee - Internship,Pros and Cons,It is a well known company and It provides you...,Some time work is with other offices which req...,none,5.0,3.0,5.0,4.0,5.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...,Iran,Tehran,Tehran
35554,35555,facebook,"Mar 7, 2018",Current Employee - Advertising,Advertising,"I like it and I think it is useful for me, i a...","Cars, fashion, actor, actress, veiw of beautif...",No lie be honest,5.0,5.0,5.0,5.0,5.0,5.0,0,https://www.glassdoor.com/Reviews/Facebook-Rev...,Iran,Tehran,Tehran
35568,35569,facebook,"Jan 28, 2018",Former Employee - 3D Modeler/Texturer,"3D artist, Modeler, texture artist",It is a very flexible company with new and cre...,I was working as a freelance artist.,none,3.0,none,none,none,none,none,0,https://www.glassdoor.com/Reviews/Facebook-Rev...,Iran,Tehran,Tehran
47447,47448,apple,"Sep 8, 2017",Current Employee - Industrial Engineer,industrial engineer,centralization innovation and ingenuity i win ...,quality control and business assurance quality...,none,5.0,3.0,none,3.0,3.0,none,0,https://www.glassdoor.com/Reviews/Apple-Review...,Iran,Kermanshah,Kermānshāh
65527,65528,microsoft,"Jan 29, 2015",Current Employee - Computer Programmer,Programming analyst and give the idea about ne...,share my ideas with the world,no cons...i love to work with computers,like to give a new application ideas,3.0,3.0,3.0,3.0,3.0,3.0,0,https://www.glassdoor.com/Reviews/Microsoft-Re...,Iran,East,Tabrīz


In [15]:
df[df.country == 'USA'].state.value_counts()

WA    11528
CA     7261
TX     1705
NY     1331
FL      775
IL      571
NJ      524
PA      505
VA      500
AZ      441
MA      421
KY      410
GA      395
TN      390
IN      317
NC      276
OR      269
MD      257
OH      254
SC      233
CO      204
NV      200
WI      166
MI      163
DC      147
DE      142
UT      131
MN      131
KS      120
CT       98
MO       84
ND       80
WV       53
OK       40
NH       37
LA       36
AL       29
HI       28
NM       19
AR       18
ME       16
NE       12
MS       11
IA       11
RI        9
PR        9
ID        8
AK        7
SD        3
WY        2
MT        2
Name: state, dtype: int64

In [16]:
df.company.value_counts()

amazon       26430
microsoft    17930
apple        12950
google        7819
facebook      1590
netflix        810
Name: company, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67529 entries, 0 to 67528
Data columns (total 19 columns):
Unnamed: 0                    67529 non-null int64
company                       67529 non-null object
dates                         67529 non-null object
job-title                     67529 non-null object
summary                       67409 non-null object
pros                          67529 non-null object
cons                          67529 non-null object
advice-to-mgmt                67232 non-null object
overall-ratings               67529 non-null float64
work-balance-stars            67529 non-null object
culture-values-stars          67529 non-null object
carrer-opportunities-stars    67529 non-null object
comp-benefit-stars            67529 non-null object
senior-mangemnet-stars        67529 non-null object
helpful-count                 67529 non-null int64
link                          67529 non-null object
country                       42444 non-null object
state   